<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
    </a>
</p>

# Final Project: Advanced SQL Techniques

## Objectives

After completing this lab, you will be able to:

*   Use joins to query data from multiple tables

*   Create and query views

*   Write and run stored procedures

*   Use transactions


## Scenario

In this project, you will work with three datasets that are available on the City of Chicago's Data Portal:

*    Socioeconomic indicators in Chicago
*    Chicago public schools
*    Chicago crime data

You must download each dataset, create a table for each one, and load the appropiate dataset through the Db2 console. If you have already completed the Hands on Lab: Joins, you can reuse the tables you created for that hands-on lab. However, you should not reuse similar tables with other names from other exercises or labs, as they may not create the correct results.

Important note:

If you have not yet downloaded the three datasets from the City of Chicago's Data Portal, created the required tables, and loaded the data, please follow the instructions in this section.



### City of Chicago Datasets
    Socioeconomic indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a "hardship index", for each Chicago community area, for the years 2008 – 2012. A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

    Chicago public schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

    Chicago crime data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

In [ ]:
import urllib.request
from urllib.request import urlopen
import ssl
import json
import csv
import sqlite3
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import csv, sqlite3

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

%load_ext sql
%sql sqlite:///RealWorldData.db

In [ ]:
df_se = pd.read_csv('https://data.cityofchicago.org/resource/jcxq-k9xf.csv')
df_se.to_sql("chicago_socioeconomic", con, if_exists='replace', index=False,method="multi")

In [ ]:
df_ps = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df_ps.to_sql("chicago_public_schools", con, if_exists='replace')

In [ ]:
df_cr = pd.read_json("https://data.cityofchicago.org/resource/ijzp-q8t2.json")
df_cr = df_cr.drop(columns = ["location"]) # need to remove any dictionaries! 
df_cr.to_sql("chicago_crimes", con, if_exists='replace', index=False)

In [ ]:
# First, find out what data is in each dataframe
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%sql SELECT * FROM chicago_socioeconomic limit 3

In [ ]:
%sql SELECT * FROM chicago_public_schools limit 3

In [ ]:
%sql SELECT * FROM chicago_crimes limit 3

Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

In [ ]:
# Both Public School and the Socioeconomic dataframes have community area name
# The public school df community area name is CAPITAL letters, and needs to be lowered.
# so the join matches!

%sql UPDATE chicago_socioeconomic SET community_area_name = LOWER(community_area_name)

%sql UPDATE chicago_public_schools SET COMMUNITY_AREA_NAME = LOWER(COMMUNITY_AREA_NAME)

In [ ]:
# Left join socioeconomic ONTO public schools

%sql SELECT chicago_public_schools.COMMUNITY_AREA_NAME, \
chicago_public_schools.AVERAGE_STUDENT_ATTENDANCE \
FROM chicago_public_schools LEFT JOIN chicago_socioeconomic \
ON chicago_public_schools.COMMUNITY_AREA_NAME = chicago_socioeconomic.community_area_name \
limit 10

Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [ ]:
%sql SELECT cc.CASE_NUMBER, cc.PRIMARY_TYPE, cp.COMMUNITY_AREA_NAME \
FROM chicago_crimes cc , chicago_public_schools cp \
LEFT JOIN chicago_public_schools \
on cp.COMMUNITY_AREA_NUMBER = cc.community_area \
WHERE cc.LOCATION_DESCRIPTION LIKE '%School%' \
LIMIT 10

Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.

In [ ]:
%sql CREATE VIEW chicago_summary(School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating) \
AS SELECT NAME_OF_SCHOOL,Safety_Icon, Family_Involvement_Icon, Environment_Icon,Instruction_Icon, Leaders_Icon, Teachers_Icon \
FROM chicago_public_schools;

In [ ]:
%sql select * from chicago_summary limit 10

Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [ ]:
%sql select School_Name, Leaders_Rating from chicago_summary limit 10

Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer. Don't forget to use the #SET TERMINATOR statement to use the @ for the CREATE statement terminator.

In [ ]:
# SET TERMINATOR @
CREATE PROCEDURE UPDATE_LEADERS_SCORE(IN in_School_ID INTEGER, IN in_Leader_Score INTEGER)
BEGIN
 UPDATE CHICAGO_PUBLIC_SCHOOLS
 SET Leaders_Score = in_Leader_Score
 WHERE School_ID = in_School_ID ;

IF in_Leader_Score >0 AND in_Leader_Score <20
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Very Weak'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 40
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Weak'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 60
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Average'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 80
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Strong'
    WHERE School_ID = in_School_ID;
ELSEIF in_Leader_Score < 100
    THEN UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Icon ='Very Strong'
    WHERE School_ID = in_School_ID;
ELSE  
   ROLLBACK;
END IF;
END@